### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import math
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [2]:
data

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2
5,0,2,2,2
6,1,2,2,2
7,0,2,2,1
8,0,1,2,2
9,1,1,1,2


In [3]:
dataStolen = data[data['Stolen?']==1]
NumStolen = len(dataStolen)
dataStolen

,Stolen?,Color,Type,Origin
0,1,1,1,1
2,1,1,1,1
4,1,2,1,2
6,1,2,2,2
9,1,1,1,2


In [4]:
dataRedStolen = dataStolen[dataStolen.Color == 1]
NumRedStolen = len(dataRedStolen)
print "P(Red|Stolen) = {0}".format(NumRedStolen * 1.0/NumStolen)

P(Red|Stolen) = 0.6


In [5]:
dataSUVStolen = dataStolen[dataStolen.Type == 2]
NumSUVStolen = len(dataSUVStolen)
print "P(SUV|Stolen) = {0}".format(NumSUVStolen * 1.0/NumStolen)

P(SUV|Stolen) = 0.2


In [6]:
dataDomesticStolen = dataStolen[dataStolen.Origin == 1]
NumDomesticStolen = len(dataDomesticStolen)
print "P(Domestic|Stolen) = {0}".format(NumDomesticStolen * 1.0/NumStolen)

P(Domestic|Stolen) = 0.4


In [7]:
dataNotStolen = data[data['Stolen?']==0]
NumNotStolen = len(dataNotStolen)
dataNotStolen

,Stolen?,Color,Type,Origin
1,0,1,1,1
3,0,2,1,1
5,0,2,2,2
7,0,2,2,1
8,0,1,2,2


In [8]:
dataRedNotStolen = dataNotStolen[dataNotStolen.Color == 1]
NumRedNotStolen = len(dataRedNotStolen)
print "P(Red|Not Stolen) = {0}".format(NumRedNotStolen * 1.0/NumNotStolen)

P(Red|Not Stolen) = 0.4


In [9]:
dataSUVNotStolen = dataNotStolen[dataNotStolen.Type == 2]
NumSUVNotStolen = len(dataSUVNotStolen)
print "P(SUV|Not Stolen) = {0}".format(NumSUVNotStolen * 1.0/NumNotStolen)

P(SUV|Not Stolen) = 0.6


In [10]:
dataDomesticNotStolen = dataNotStolen[dataNotStolen.Origin == 1]
NumDomesticNotStolen = len(dataDomesticNotStolen)
print "P(Domestic|Not Stolen) = {0}".format(NumDomesticNotStolen * 1.0/NumNotStolen)

P(Domestic|Not Stolen) = 0.6


b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 
Please perform all the necessary computations "by hands" rather than using python code.

In [11]:
#For compare proportions (not actual probabilities):
#Since P(Stolen) = P(NotStole) = 0.5, not because it is not known, but because it is half for each in initial table 
#P(NotStolen|{Red|SUV|Domestic}) = P(Red|Not Stolen) * P(Domestic|Not Stolen) * P(SUV|Not Stolen) * P(Not Stolen)
print "P(NotStolen|{Red|SUV|Domestic}) = %s" % (0.4*0.6*0.6*0.5)
#P(Stolen|{Red|SUV|Domestic}) = P(Red|Stolen) * P(Domestic|Stolen) * P(SUV|Stolen) * P(Stolen)
print "P(Stolen|{Red|SUV|Domestic}) = %s" %(0.4*0.2*0.6*0.5)

P(NotStolen|{Red|SUV|Domestic}) = 0.072
P(Stolen|{Red|SUV|Domestic}) = 0.024


So, a vehicle Red, SUV and Domestic is more likely to be NOT STOLEN since 0.072 > 0.024

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

In [12]:
data_train=pd.read_csv("dia_train.csv") 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("dia_test.csv")
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

In [13]:
#Use class functions
def trainNaiveBayes(trainData):
    #training Gausian Naive Bayes Classifier
    tY=trainData.loc[:,trainData.columns[0]]
    ind1=tY==0
    ind2=tY==1
    dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
    #estimate priors
    dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
    dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
    #estimate sample distribution paramters for p(xi|y=b)
    for i in trainData.columns[1:]:
        dp.loc['mu1',i]=(trainData[i][ind1]).mean()
        dp.loc['sigma1',i]=(trainData[i][ind1]).std()
        dp.loc['mu2',i]=(trainData[i][ind2]).mean()
        dp.loc['sigma2',i]=(trainData[i][ind2]).std()
    return dp
def classifyNaiveBayes(classData,dp):
    #classifying using trained Gausian Naive Bayes Classifier
    Y=classData.loc[:,classData.columns[0]]*0
    for j in classData.index:
        #start from the priors
        P1=dp[classData.columns[0]]['mu1'];
        P2=dp[classData.columns[0]]['mu2'];
        #multiply by conditional probability densities p(xi|y=b)
        for i in classData.columns[1:]:
            if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
                P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
            else:
                P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
            if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
                P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
            else:
                P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
        Y[j]=int(P2>P1)
 

    return Y

In [14]:
dataTrain=pd.DataFrame({'Y':y_train,'t_pre':X_train.t_pre,'glu':X_train.glu,'blood_p':X_train.blood_p,'triceps':X_train.triceps,'serum':X_train.serum,'b_m':X_train.b_m,'pedigree_f':X_train.pedigree_f,'age':X_train.age})
dataTrain.head()

,Y,age,b_m,blood_p,glu,pedigree_f,serum,t_pre,triceps
0,0,24,31.2,64,158,0.295,387,3,13
1,0,21,35.8,64,84,0.545,66,0,22
2,0,48,20.8,72,120,0.733,56,9,22
3,0,27,28.4,76,110,0.118,100,4,20
4,1,28,32.9,66,100,0.867,90,2,20


In [15]:
dataTest=pd.DataFrame({'Y':y_test,'t_pre':X_test.t_pre,'glu':X_test.glu,'blood_p':X_test.blood_p,'triceps':X_test.triceps,'serum':X_test.serum,'b_m':X_test.b_m,'pedigree_f':X_test.pedigree_f,'age':X_test.age})
dataTest.head()

,Y,age,b_m,blood_p,glu,pedigree_f,serum,t_pre,triceps
0,0,21,28.1,66,89,0.167,94,1,23
1,1,33,43.1,40,137,2.288,168,0,35
2,1,53,30.5,70,197,0.158,543,2,45
3,1,59,30.1,60,189,0.398,846,1,23
4,0,33,43.3,30,103,0.183,83,1,38


In [16]:
dp=trainNaiveBayes(dataTrain) #train the Gausian Naive Bayes Classifier
dp.head()

,Y,age,b_m,blood_p,glu,pedigree_f,serum,t_pre,triceps
mu1,0.6991525,28.39394,31.70909,69.20606,111.4667,0.4686848,127.0061,2.690909,27.2
sigma1,NaN,8.537362,6.337613,11.71329,24.69189,0.2917503,91.48614,2.610061,10.43692
mu2,0.3008475,35.78873,35.22394,74.56338,144.1408,0.6390423,209.2113,4.070423,33.47887
sigma2,NaN,10.26355,6.258491,13.79931,30.62646,0.439042,126.921,3.51862,9.762697


b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

In [17]:
C=classifyNaiveBayes(dataTest,dp) #classify test set
#classification accuracy (over test set)
100.0*sum(C==y_test)/len(C)

79.11392405063292

In [18]:
#also report accuracy of the classifier over the training - it is slightly higher, although not that higher
Ct=classifyNaiveBayes(dataTrain,dp)
100.0*sum(Ct==y_train)/len(Ct)

76.27118644067797

### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.


In [19]:
data_train=pd.read_csv("EM_train.csv", usecols=range(1,5))
y_Label_train=data_train.iloc[:,0] 
X_Label_train=data_train.iloc[:,1:] 

data_test=pd.read_csv("EM_test.csv", usecols=range(1,5))
y_Label_test=data_test.iloc[:,1]
X_Label_test=data_test.iloc[:,2:]

data_Unlabel=pd.read_csv("EM_Unlabel.csv", usecols=range(1,4))
X_Unlabel=data_Unlabel.iloc[:,1:]

In [20]:
def trainNaiveBayesDiscrete(trainData):
    #training discrete Naive Bayes Classifier
    tY=trainData.loc[:,trainData.columns[0]]
    m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
    #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
    dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
    #split the training data between two labels
    ind1=tY==0
    ind2=tY==1
    #estimate P(y=b)  
    dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
    dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
    #estimate conditional probabilities P(x|y=b)
    for j in trainData.columns[1:]:
        for i in range(1,m+1):
            dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
            dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
    return dp
def classifyNaiveBayesDiscrete(classData,dp):
    #classifying using trained discrete Naive Bayes Classifier
    Y=classData[classData.columns[0]]*0 #initialize the empty array 
    for i in classData.index: #for al records to classify
        #start with the priors
        P1=dp[0][classData.columns[0]][1]; 
        P2=dp[1][classData.columns[0]][1];
        #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
        for j in classData.columns[1:]:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
    return Y

#### Questions:
a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

In [21]:
dp=trainNaiveBayesDiscrete(data_train)
#Classification over the Test Set
C=classifyNaiveBayesDiscrete(data_test,dp)
100.0*sum(C==data_test.y)/len(C)

63.888888888888886

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook).

In [22]:
def EM(X_Label,y_Label,X_Unlabel,dp,PrintLikelihood=True):
    t = 0  
    haslabels=len(y_Label)>0

    while True:
        t = t + 1

        classData=X_Unlabel
        # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
        # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

        #for y=1 and y=2

        p_x_1=[] #unnormalized P(y=1|x)
        p_x_2=[] #unnormalized P(y=2|x)
        cols=dp[0].columns

        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            p_x_1.append(P1)
            p_x_2.append(P2)

        #Rescale p_x_1 and p_x_2:
        summ=np.asarray(p_x_1)+np.asarray(p_x_2)
        p_x_1_s=np.asarray(p_x_1)/summ
        p_x_2_s=np.asarray(p_x_2)/summ
        inds_1 = np.where(np.isnan(p_x_1_s))
        inds_2 = np.where(np.isnan(p_x_2_s))
        p_x_1_s[inds_1]=0.5
        p_x_2_s[inds_2]=0.5
        #Now let's calculate P(y=1) and P(y=2)
        p_1=p_x_1_s.sum()/len(p_x_1_s)
        p_2=p_x_2_s.sum()/len(p_x_2_s)


        #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
        m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

        #create output data structure for the probabilities - new iteration
    
        dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

        #P(y=b)  
        dp1[0][cols[0]][1]=p_1
        dp1[1][cols[0]][1]=p_2


        #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

        temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
        temp=pd.DataFrame(temp)
        if haslabels:
            temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
            temp_l=pd.DataFrame(temp_l)
            pd.concat([temp,temp_l])
   

        for j in range(1,len(dp[0].T)):
            for i in range(len(dp[0])):

                dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
                dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
        # Now we use dp to decide whether to continue our iterations
    
        #where_nan_dp1 = np.isnan(dp1) 
        #dp1[where_nan_dp1] = 0.0
        #where_nan_dp = np.isnan(dp) 
        #dp[where_nan_dp] = 0.0
        #print sum(dp1[1] - dp[1])
        if (sum(np.sum((dp1[0]-dp[0])**2))+sum(np.sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
            break
        else: 
            dp=dp1  #save new dp and perform next iteration

        
        ###############################################################################################
        #Calculate the log-likelihood
        
            L=0
        
            for i in classData.index:
                P1=dp[0][cols[0]][1];
                P2=dp[1][cols[0]][1];
                for j in classData.columns:
                    P1=P1*dp[0][j][classData[j][i]]
                    P2=P2*dp[1][j][classData[j][i]]
                temp=math.log(P1+P2)
                L=L+temp
            if haslabels:    
                for i in X_Label.index:
                    yi=y_Label[i]
                    P=dp[yi][cols[0]][1];
                    for j in X_Label.columns:
                        P=P*dp[yi][j][X_Label[j][i]]
                    L=L+math.log(P)
        
            if PrintLikelihood:
                print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
    return dp

In [23]:
dp2 = EM(X_Label_train, y_Label_train, data_Unlabel, dp)

C=classifyNaiveBayesDiscrete(data_test,dp2) 
#Classifying
100.0*sum(C==data_test.y)/len(C)

Iteration 1: log maximum liklihood = -590.452072842
Iteration 2: log maximum liklihood = -572.346722303
Iteration 3: log maximum liklihood = -559.190769042
Iteration 4: log maximum liklihood = -556.633109435
Iteration 5: log maximum liklihood = -556.067798435


94.44444444444444

### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:


In [24]:
data=pd.read_csv("EM_Cluster.csv")
X=data.iloc[:,1:]

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

In [25]:
#Create Artificial Y
np.random.seed(2016)
Y = np.random.randint(0,2,len(X))
#Attach to X
X.insert(0, 'Y', Y)

In [26]:
dp=trainNaiveBayesDiscrete(X)
dp2 = EM([], [], X.iloc[:,1:], dp)
C=classifyNaiveBayesDiscrete(X,dp2)
#Reportin vector of cluster number for each data record
report = pd.DataFrame({'C':C,'0':X['0'],'1':X['1'],'2':X['2']})
report

Iteration 1: log maximum liklihood = -555.253482473


,0,1,2,C
0,5,6,6,0
1,4,6,6,0
2,1,5,5,1
3,1,3,4,0
4,5,6,2,0
5,2,5,4,1
6,3,3,3,0
7,4,4,5,1
8,5,5,5,1
9,1,1,1,1



b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

In [32]:
reports = pd.DataFrame()
StayWhile = True
for i in range(10):
    data=pd.read_csv("EM_Cluster.csv")
    X=data.iloc[:,1:]

    #Create Artificial Y
    Y = np.random.randint(0,2,len(X))
    #Attach to X
    X.insert(0, 'Y', Y)

    dp=trainNaiveBayesDiscrete(X)
    dp2 = EM([], [], X.iloc[:,1:], dp,False)
    C=classifyNaiveBayesDiscrete(X,dp2)
    reports[str(i)] = C
# Analyze 
analyzed = 1 - np.abs(np.diff(reports))
analyzed=pd.DataFrame(analyzed)
analyzed

,0,1,2,3,4,5,6,7,8
0,0,1,0,0,0,0,1,1,1
1,0,1,0,0,0,0,1,1,1
2,0,1,0,0,0,1,0,1,1
3,0,1,0,0,0,1,0,1,1
4,0,1,0,0,0,0,1,1,1
5,0,1,0,0,0,1,0,1,1
6,0,1,0,0,0,1,0,1,1
7,0,1,0,0,0,1,0,1,1
8,0,1,0,0,0,1,0,1,1
9,0,1,0,0,0,0,1,1,1


In [33]:
#Estimate Average mean and STD
means = reports.mean(axis=1)
print "means:"
means

means:


0      0.7
1      0.7
2      0.6
3      0.4
4      0.7
5      0.6
6      0.4
7      0.6
8      0.6
9      0.3
10     0.3
11     0.3
12     0.3
13     0.7
14     0.7
15     0.7
16     0.7
17     0.7
18     0.3
19     0.4
20     0.4
21     0.6
22     0.3
23     0.3
24     0.7
25     0.3
26     0.6
27     0.7
28     0.6
29     0.3
      ... 
75     0.6
76     0.6
77     0.4
78     0.7
79     0.3
80     0.3
81     0.4
82     0.6
83     0.3
84     0.4
85     0.7
86     0.7
87     0.7
88     0.7
89     0.6
90     0.3
91     0.7
92     0.4
93     0.5
94     0.6
95     0.3
96     0.7
97     0.4
98     0.4
99     0.7
100    0.3
101    0.4
102    0.3
103    0.6
104    0.6
dtype: float64

In [34]:
stds = reports.std(axis=1)
print "standard errors:"
print stds

standard errors:
0      0.483046
1      0.483046
2      0.516398
3      0.516398
4      0.483046
5      0.516398
6      0.516398
7      0.516398
8      0.516398
9      0.483046
10     0.483046
11     0.483046
12     0.483046
13     0.483046
14     0.483046
15     0.483046
16     0.483046
17     0.483046
18     0.483046
19     0.516398
20     0.516398
21     0.516398
22     0.483046
23     0.483046
24     0.483046
25     0.483046
26     0.516398
27     0.483046
28     0.516398
29     0.483046
         ...   
75     0.516398
76     0.516398
77     0.516398
78     0.483046
79     0.483046
80     0.483046
81     0.516398
82     0.516398
83     0.483046
84     0.516398
85     0.483046
86     0.483046
87     0.483046
88     0.483046
89     0.516398
90     0.483046
91     0.483046
92     0.516398
93     0.527046
94     0.516398
95     0.483046
96     0.483046
97     0.516398
98     0.516398
99     0.483046
100    0.483046
101    0.516398
102    0.483046
103    0.516398
104    0.516398
dtype: 